In [1]:
# I want to create a rectangular box around a center.
# that will allow me to display the detection and tracking success in ChimeraX over time in a movie. Nice!

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from numpy import genfromtxt


import pandas as pd

from mpl_toolkits import mplot3d
#%matplotlib notebook



#import cv2
import pims
from PIL import Image

import skimage

import json



# cube as a mesh

In [3]:
def getCubeMeshIndexes(cubeSize = 5, center=[0,0,0]):
# this is 5 on each side of the center
#center=np.array([224.29615185,362.37867407,45.60376889])
    indexes = []
    limits = np.array([-cubeSize,+cubeSize])
    for i in range(limits[0]-2,limits[1]+2):
        for j in  range(limits[0]-2,limits[1]+2):
            for k in range(limits[0]-2,limits[1]+2):
                value = 0
                if i in (limits+center[0]) and j in (limits+center[1]) and ((k > (limits+center[2])[0]) and (k < (limits+center[2])[1])) :
    #                print(limits+centerOfImage[0]+coord[0])
    #                print(limits+centerOfImage[1]+coord[1])
                    indexes.append([i,j,k])
                if j in limits+center[1] and k in limits+center[2] and ((i > (limits+center[0])[0]) and (i < (limits+center[0])[1])) :
                    indexes.append([i,j,k])
                if k in limits+center[2] and i in limits+center[0] and ((j > (limits+center[1])[0]) and (j < (limits+center[1])[1])) :
                    indexes.append([i,j,k])
    return(indexes)

# one cube for every detection

In [4]:
inputParameters = pd.read_csv('../_inputParameters.csv',names=['key','value'])
inputParameters.style.set_properties(**{'text-align': 'left'})
#inputParameters

,key,value
0,inputDataFolder,/Users/johannesschoeneberg/Desktop/PostDoc/drubin_lab/organoids_lattice_data/data_/2016_04_25Daphne/Sample3_CD_10ulMG/Position5_Basal_apical_ObjScan/matlab_decon
1,outputDataFolder,/Users/johannesschoeneberg/Desktop/PostDoc/drubin_lab/organoids_lattice_data/data_/2016_04_25Daphne/Sample3_CD_10ulMG/Position5_Basal_apical_ObjScan/matlab_decon/output
2,movieLength,70
3,allowedMaxNumDetectionsPerFrame,3000
4,framerate_msec,5689
5,ch0_outputDataFolder,output_488
6,ch0_uniqueFilenameString,488nm_0
7,ch0_detectionFilename,Detection3D.mat
8,ch0_trackingFilename,trackedFeatures.mat
9,ch0_trackingCsvFilename,trackedFeatures.csv


In [5]:
def getInputParameter(inputParametersPandas,key):
    #this locates the row, gets the result out of its array form and strips whitespaces away
    return (((inputParametersPandas.loc[inputParametersPandas['key'] == key]).values)[0,1]).strip()

In [6]:
inputDataFolder = getInputParameter(inputParameters,"inputDataFolder")
outputDataFolder = getInputParameter(inputParameters,"outputDataFolder")
ch0_outputDataFolder  = getInputParameter(inputParameters,"ch0_outputDataFolder")
ch0_trackingCsvFilename = getInputParameter(inputParameters,"ch0_trackingCsvFilename")
ch1_outputDataFolder  = getInputParameter(inputParameters,"ch1_outputDataFolder")
ch1_trackingCsvFilename = getInputParameter(inputParameters,"ch1_trackingCsvFilename")

ch0_uniqueFilenameString = getInputParameter(inputParameters,"ch0_uniqueFilenameString")
ch1_uniqueFilenameString = getInputParameter(inputParameters,"ch1_uniqueFilenameString")
#trackColor = getInputParameter(inputParameters,"trackColor")
#trackingBildFilename = getInputParameter(inputParameters,"trackingBildFilename")
#framerate_msec = float(getInputParameter(inputParameters,"framerate_msec"))
movieLength = int(getInputParameter(inputParameters,"movieLength"))
#print(trackColor)






In [7]:
folder = outputDataFolder+'/'+ch1_outputDataFolder+'/'

data = pd.read_csv(folder+'puncta_1.csv',header=0)
data.columns = ["x","y","z","A"]
#allDetectedPunctaInFirstFrame = data[['x','y','z']].values.astype(int)-np.array([1,1,1])
#allDetectedPunctaInFirstFrame = data[['y','z','x']].values.astype(int)-np.array([1,1,1])
#allDetectedPunctaInFirstFrame = data[['z','x','y']].values.astype(int)-np.array([1,1,1])

#### WARNING ####

# this is really weird!
# for some reason, I have to invert all the coordinates to display them again...

allDetectedPunctaInFirstFrame = data[['z','y','x']].values.astype(int)-np.array([1,1,1])
print(np.sort(allDetectedPunctaInFirstFrame))

[[ 31 277 297]
 [ 32 306 321]
 [ 34 314 373]
 ..., 
 [145 196 248]
 [196 288 701]
 [196 342 701]]


In [9]:
def getRidOfNegativeIndexes(indexesArray):
    newIndexes = []
    for index in indexesArray:
        if(np.min(index)>=0):
            newIndexes.append(index)
    return(np.array(newIndexes))

In [10]:
def getIndexesWhereAllArrayEntriesArePositive(indexesArray):
    newIndexes = []
    for i in range(0,len(indexesArray)):
        index=indexesArray[i]
        if(np.min(index)>=0):
            newIndexes.append(i)
    return(np.array(newIndexes))

In [11]:
# this works. Nice!

margin = 110
margin = 0
imageSizeOriginal =(201, 704, 600) # this is the size of one lattice frame
imageSize = np.array((imageSizeOriginal[0]+margin,imageSizeOriginal[1]+margin,imageSizeOriginal[2]+margin))
#imageSize =(1001, 1000, 1000) # this is the size of one lattice frame

centerOfImage = np.array([imageSize[0]//2,imageSize[1]//2,imageSize[2]//2])-np.array([1,1,1])
centerOfImage = np.array([imageSize[0]//2,imageSize[1]//2,imageSize[2]//2])-np.array([1,1,1])

boxCenters = allDetectedPunctaInFirstFrame

image = np.zeros(imageSize)
print(imageSize)

meshIndexes = getCubeMeshIndexes()
meshIndexesAdjustedArray = []
counter = 0

for boxCenter in boxCenters:
    
    
    if counter%100 == 0:
        print(counter)
        
        
 #   if(boxCenter[1]>imageSize[1]):
 #       boxCenter[1]=boxCenter[1]-imageSize[1]
 #       print(boxCenter)


    # I run into problems if the box Center leads to negative values -> prune them
    meshIndexesAdjusted = meshIndexes+boxCenter
    if np.min(meshIndexesAdjusted)<0:
        meshIndexesAdjusted = meshIndexesAdjusted[getIndexesWhereAllArrayEntriesArePositive(meshIndexesAdjusted)]
    meshIndexesAdjustedArray.append(meshIndexesAdjusted)
    
    
    # I run into problems if the box center leads to indexes that are larger than the image -> prune them
    boolArray = boxCenter+np.array([6,6,6]) >= imageSize
    if(True in boolArray):
        # these values are larger than the image size, subtract the image size first, get negative values
        tmp = imageSize-np.array([1,1,1]) - meshIndexesAdjusted
        # add the image size back after gotten rid of the negative ones
        idx=getIndexesWhereAllArrayEntriesArePositive(tmp)
        if(len(idx)==0):
            print('weird box center (not in image), excluded from display: ')
            print(boxCenter)
            print(imageSize)
            meshIndexesAdjusted = []
        else:
            meshIndexesAdjusted = meshIndexesAdjusted[idx]
            

    
    
    
    #print('newCoordsShape',newCoords.shape)
    #print(newCoords)
    #print(len(newCoords))
    #print(image.shape)
    
    #image[newCoords]=10
    #print(newCoords)
    #print('test')
    #print(newCoords)
    
        
    
    
    for index in meshIndexesAdjusted:
        #print(index)
    #    print('index',index)
        image[index[0],index[1],index[2]]=10
    
    #print(image)
    
    
    counter = counter+1

data = image.astype('uint16')
metadata = dict(microscope='joh', shape=data.shape, dtype=data.dtype.str)
metadata = json.dumps(metadata)
print(metadata)
skimage.external.tifffile.imsave('./boxes_560firstFrame_zyx_smaller.tif', data, description=metadata)


[201 704 600]
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
{"microscope": "joh", "shape": [201, 704, 600], "dtype": "<u2"}
